In [1]:
import time

import gizmo_analysis as gizmo
import gc_utils  # type: ignore
import h5py
import numpy as np
import pandas as pd
import utilities as ut

In [2]:
## function inputs
sim = "m12i"
# snapshot = 600
snapshot = 214  # most gcs
main_halo_tid = 25236877

# sim_dir = "../../simulations/"
sim_dir = "../../../simulations/"  # went up an extra level as in notebook folder
data_dict = {}

it_lst = [0]

fire_dir = sim_dir + sim + "/" + sim + "_res7100/"

# think about species, for basic kinematics just upload star and dark
part = gizmo.io.Read.read_snapshots(["dark", "star"], "index", snapshot, fire_dir, assign_hosts_rotation=True)


# in utilities.simulation.Snapshot():
* reading:  simulations/m12i/m12i_res7100/snapshot_times.txt

  using snapshot index = 214, redshift = 1.500


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  simulations/m12i/m12i_res7100/output/snapdir_214/snapshot_214.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 68214754 particles
    star      (id = 4): 2309196 particles
    blackhole (id = 5): 0 particles

* reading the following
  species: ['dark', 'star']

* reading particles from:
    snapshot_214.0.hdf5
    snapshot_214.1.hdf5
    snapshot_214.2.hdf5
    snapshot_214.3.hdf5

* reading cosmological parameters from:  simulations/m12i/m12i_res7100/initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties


# in gizmo_analysis.gizmo_track.ParticleCoordinate():
  read 1 host (position, velocity, principal axes) from:  simulations/m12i/m

In [3]:
proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
proc_data = h5py.File(proc_file, "r")  # open processed data file

In [4]:
halt = gc_utils.get_halo_tree(sim, sim_dir, assign_hosts_rotation=False)

Retrieving Halo Tree.....................: 100%|████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.28s/it]



# in utilities.simulation.Snapshot():
* reading:  simulations/m12i/m12i_res7100/snapshot_times.txt

  using snapshot index = 214, redshift = 1.500


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  simulations/m12i/m12i_res7100/output/snapdir_214/snapshot_214.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 68214754 particles
    star      (id = 4): 2309196 particles
    blackhole (id = 5): 0 particles

* reading the following
  species: ['star']

* reading particles from:
    snapshot_214.0.hdf5
    snapshot_214.1.hdf5
    snapshot_214.2.hdf5
    snapshot_214.3.hdf5

* reading cosmological parameters from:  simulations/m12i/m12i_res7100/initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties


# in gizmo_analysis.gizmo_track.ParticleCoordinate():
  read 1 host (position, velocity, principal axes) from:  simulations/m12i/m12i_res7

In [5]:
def remove_duplicates_with_report(arr):
    arr = np.array(arr)  # Convert the list to a numpy array
    unique_elements, counts = np.unique(arr, return_counts=True)  # Get unique elements and their counts
    duplicates = unique_elements[counts > 1]  # Duplicates are those that appear more than once
    unique_list = unique_elements.tolist()  # Convert unique elements back to a list

    return unique_list, duplicates.tolist()  # Return unique and duplicate lists

In [6]:
# id_idx_map, gc_id_snap, ptype_snap = gc_utils.create_gc_part_idx_dict(part, proc_data, it, snapshot)

it = 1
it_id = gc_utils.iteration_name(it)
snap_id = gc_utils.snapshot_name(snapshot)

gc_id_snap = proc_data[it_id]["snapshots"][snap_id]["gc_id"][()]

ptype_byte_snap = proc_data[it_id]["snapshots"][snap_id]["ptype"]
ptype_snap = [ptype.decode("utf-8") for ptype in ptype_byte_snap]

# Step 1: group GCs by particle type
gc_by_ptype = {}
gc_by_ptype["star"] = []
gc_by_ptype["dark"] = []

for gc, ptype in zip(gc_id_snap, ptype_snap):
    gc_by_ptype[ptype].append(gc)

# Step 2: for each ptype, build a small dict: gc_id → index
id_idx_map = {}

for ptype, gc_ids in gc_by_ptype.items():
    ids = part[ptype]["id"]  # potentially millions of entries
    # gc_ids = np.array(gc_ids)  # small subset

    # Check which of these are in the main list
    mask = np.isin(ids, gc_ids)
    idxs = np.nonzero(mask)[0]
    found_ids = ids[idxs]

    # Build small, efficient lookup: GC ID → array index
    id_idx_map[ptype] = dict(zip(found_ids, idxs))

    # concerned abour duplciate star ids
    if ptype == "star":
        _, duplicates_ids = remove_duplicates_with_report(found_ids)

In [7]:
duplicates_ids

[42251338, 48721435]

In [8]:
i = 0
main_halo_tid = 25236877

In [9]:
ana_mask = proc_data[it_id]["source"]["analyse_flag"][()] == 1
gc_mask = proc_data[it_id]["source"]["gc_id"][()] == duplicates_ids[i]
snap_form = proc_data[it_id]["source"]["snap_zform"][ana_mask & gc_mask][0]
snap_form

210

In [10]:
# get closest next public snapshot (previously calculated)
pubsnap_form = proc_data[it_id]["source"]["pubsnap_zform"][ana_mask & gc_mask][0]
pubsnap_form

214

In [11]:
group_id = np.abs(proc_data[it_id]["source"]["group_id"][ana_mask & gc_mask][0])
group_id

0

In [12]:
if group_id == 0:
    halo_tid = main_halo_tid
else:
    halo_tid = group_id

halo_tid_snap = gc_utils.get_halo_prog_at_snap(halt, halo_tid, pubsnap_form)
halo_tid_snap

6279613

In [13]:
halo_idx = np.where(halt["tid"] == halo_tid_snap)[0][0]
snapcheck = halt["snapshot"][halo_idx]
snapcheck

214

In [14]:
halo_pos = halt["position"][halo_idx]
halo_pos

array([42694.8  , 42784.387, 45005.57 ], dtype=float32)

In [15]:
part_form = gizmo.io.Read.read_snapshots(["star"], "index", pubsnap_form, fire_dir)

In [16]:
part_idxs = np.where(part["star"]["id"] == duplicates_ids[i])[0]
part_snapform = part["star"].prop("form.snapshot", part_idxs)

idx = np.where(part_snapform == snap_form)[0]
idx

array([ 5, 13, 14])

In [17]:
part_form_idxs = np.where(part_form["star"]["id"] == duplicates_ids[i])[0]
part_snapform = part_form["star"].prop("form.snapshot", part_idxs)
part_snapform

array([209, 207, 214, 205, 212, 210, 211, 214, 212, 209, 205, 209, 214,
       210, 210, 212, 211, 205, 207], dtype=int32)

In [18]:
idxs = np.where(part_snapform == snap_form)[0]
part_interest_idxs = part_form_idxs[idxs]
part_interest_idxs

array([ 524234, 1450863, 1476832])

In [19]:
# get public snapshot at snapshot before formation

public_snapshot_file = sim_dir + "snapshot_times_public.txt"
pub_data = pd.read_table(public_snapshot_file, comment="#", header=None, sep=r"\s+")
pub_data.columns = [
    "index",
    "scale_factor",
    "redshift",
    "time_Gyr",
    "lookback_time_Gyr",
    "time_width_Myr",
]
pub_snaps = np.array(pub_data["index"], dtype=int)

preform_snap_idx = np.where(pub_snaps == pubsnap_form)[0][0] - 1
preform_snap = pub_snaps[preform_snap_idx]
preform_snap

172

In [20]:
part_z0 = gizmo.io.Read.read_snapshots(["star", "gas"], "index", 600, fire_dir)

part_form = gizmo.io.Read.read_snapshots(
    ["star", "gas"], "index", pubsnap_form, fire_dir, assign_pointers=True
)
part_preform = gizmo.io.Read.read_snapshots(
    ["star", "gas"], "index", preform_snap, fire_dir, assign_pointers=True
)

In [21]:
pointers = part_form.Pointer.get_pointers(
    species_name_from="star", species_names_to=["star", "gas"], forward=True
)

# indices_at_form = part_interest_idxs
indices_at_form = part_interest_idxs
indices_at_z0 = pointers["index"][indices_at_form]
species_at_z0 = pointers["species"][indices_at_form]
indices_at_form, indices_at_z0, species_at_z0

(array([ 524234, 1450863, 1476832]),
 array([8921273, 3112214, 6912713], dtype=int32),
 array(['star', 'star', 'star'], dtype='<U4'))

In [22]:
part_form["star"]["id"][indices_at_form]

array([42251338, 42251338, 42251338], dtype=uint32)

In [23]:
part_z0["star"]["id"][indices_at_z0]

array([42251338, 42251338, 42251338], dtype=uint32)

In [53]:
part_at_z1 = gizmo.io.Read.read_snapshots("star", "redshift", 1, fire_dir, assign_pointers=True)
part_at_z2 = gizmo.io.Read.read_snapshots("star", "redshift", 2, fire_dir, assign_pointers=True)

In [54]:
part_at_z2.Pointer.add_intermediate_pointers(part_at_z1.Pointer)

In [ ]:
pointers = part_at_z2.Pointer.get_pointers(
    species_name_from="star", species_names_to="star", intermediate_snapshot=True
)

star_indices_at_z1 = np.array([2, -1, -2])
star_indices_at_z2 = pointers[star_indices_at_z1]

In [49]:
part_at_z1["star"]["id"][star_indices_at_z1]

array([31374153, 47313841, 16317647], dtype=uint32)

In [50]:
part_at_z2["star"]["id"][star_indices_at_z2]

array([31374153, 47313841, 16317647], dtype=uint32)

In [57]:
pointers = part_at_z2.Pointer.get_pointers(
    species_name_from="star", species_names_to="star", intermediate_snapshot=True, forward=True
)

star_indices_at_z2 = np.array([2, -1, -2])
star_indices_at_z1 = pointers[star_indices_at_z2]

In [59]:
part_at_z2["star"]["id"][star_indices_at_z2]

array([24915325,  8138926,  7343139], dtype=uint32)

In [58]:
part_at_z1["star"]["id"][star_indices_at_z1]

array([24915325,  8138926,  7343139], dtype=uint32)

In [24]:
#########################################

In [27]:
from GC_formation_model import astro_utils

In [29]:
cosmo = astro_utils.cosmo(h=0.702, omega_baryon=0.0455, omega_matter=0.272)

In [48]:
a_s = part_form["star"]["form.scalefactor"][indices_at_form]
a_s

array([0.39315334, 0.39361042, 0.3927175 ], dtype=float32)

In [51]:
part_form["star"].prop("form.time")[indices_at_form]

array([4.27808112, 4.28518091, 4.27131362])

In [52]:
part_form["star"].prop("age")[indices_at_form]

array([0.10659583, 0.09949603, 0.11336333])

In [47]:
scale_a = 1 / (1 + part_form.Snapshot["redshift"][snap_form])
scale_a

0.3936507996392845

In [59]:
x = part_form["star"]["position"][part_interest_idxs][:, 0] - halo_pos[0]
y = part_form["star"]["position"][part_interest_idxs][:, 1] - halo_pos[1]
z = part_form["star"]["position"][part_interest_idxs][:, 2] - halo_pos[2]

r2_s = (x**2 + y**2 + z**2) * (scale_a / cosmo.h) ** 2
r = np.sqrt(r2_s)
r

array([ 1.8170525 ,  1.95870109, 12.46928377])

In [61]:
z_s = -1 + 1.0 / a_s
z_s

array([1.5435369, 1.5405831, 1.5463595], dtype=float32)

In [63]:
t_s = np.array(cosmo.cosmicTime(z_s, units="Gyr"))
t_s

array([4.28945631, 4.29657522, 4.28267111])

In [65]:
t_form = cosmo.cosmicTime(part_form.Snapshot["redshift"][snap_form], units="Gyr")
t_form

4.2972041827863645

In [67]:
t_before = cosmo.cosmicTime(part_form.Snapshot["redshift"][snap_form] - 1, units="Gyr")
t_before

8.463784525748876

In [68]:
dt = t_form - t_before
dt

-4.1665803429625115

In [71]:
t_s < t_form

array([ True,  True,  True])

In [75]:
max_lag_ratio = 1.0
t_lag = 0.01

t_s > t_form - max(dt * max_lag_ratio, t_lag)

array([ True,  True, False])

In [76]:
qlt = (t_s > (t_form - t_lag)).astype(int) + 1
qlt

array([2, 2, 1])